In [2]:
!apt-get install swig -y  # c or c++ -> python, ruby , etc ...
!pip install Cython numpy # 파이썬 코드를 C언어로 컴파일함
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
!sudo apt-get install build-essential swig

!pip install auto-sklearn==0.10.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   209  100   209    0     0   1548      0 --:--:-- --:--:-- --:--:--  1548
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-sklearn 0.10.0 requires scikit-learn<0.23,>=0.22.0, but you have scikit-learn 0.24.2 which is inco

In [32]:
import pandas as pd
import sklearn.model_selection
import autosklearn.classification


In [5]:
Data = pd.read_csv('/content/drive/MyDrive/인공지능 빅데이터/3주차/data/winequality-red.csv')
Data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [9]:
Data.notna().sum()#결측확인

fixed acidity           1599
volatile acidity        1599
citric acid             1599
residual sugar          1599
chlorides               1599
free sulfur dioxide     1599
total sulfur dioxide    1599
density                 1599
pH                      1599
sulphates               1599
alcohol                 1599
quality                 1599
dtype: int64

In [27]:
Data_x = Data.iloc[:,0:11].to_numpy()
Data_y = Data.iloc[:,11].to_numpy()

In [30]:
print(Data_x)
print(Data_y)

[[ 7.4    0.7    0.    ...  3.51   0.56   9.4  ]
 [ 7.8    0.88   0.    ...  3.2    0.68   9.8  ]
 [ 7.8    0.76   0.04  ...  3.26   0.65   9.8  ]
 ...
 [ 6.3    0.51   0.13  ...  3.42   0.75  11.   ]
 [ 5.9    0.645  0.12  ...  3.57   0.71  10.2  ]
 [ 6.     0.31   0.47  ...  3.39   0.66  11.   ]]
[5 5 5 ... 6 5 6]


In [50]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(Data_x, Data_y,random_state=4)
print(x_train.shape)
print(y_train.shape)
print('--------------------------------------------------')
print(x_test.shape)
print(y_test.shape)


(1199, 11)
(1199,)
--------------------------------------------------
(400, 11)
(400,)


In [58]:
automl = autosklearn.classification.AutoSklearnClassifier(#시간제약상 최소화
    time_left_for_this_task=300,
    per_run_time_limit=30
)
automl.fit(x_train, y_train)
print('-------------------------------------------------------------------------------------------------------------------')
print(automl.show_models())

-------------------------------------------------------------------------------------------------------------------
[(0.700000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'adaboost', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'none', 'feature_preprocessor:__choice__': 'random_trees_embedding', 'classifier:adaboost:algorithm': 'SAMME', 'classifier:adaboost:learning_rate': 0.6255555220658239, 'classifier:adaboost:max_depth': 6, 'classifier:adaboost:n_estimators': 446, 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.008833097306364968, 'feature_preprocessor:random_trees_embedding:bootstrap': 'Fal

In [57]:
automl_predict=automl.predict(x_test)
sklearn.metrics.accuracy_score(y_test, automl_predict)

0.665

In [59]:
automl_ensemble_nbest_80 = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,
    per_run_time_limit=30,
    ensemble_nbest=80
)
automl_ensemble_nbest_80.fit(x_train, y_train)
print('-------------------------------------------------------------------------------------------------------------------')
print(automl_ensemble_nbest_80.show_models())

[WARNING] [2021-07-20 04:26:18,270:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 40
[WARNING] [2021-07-20 04:27:46,202:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 20
-------------------------------------------------------------------------------------------------------------------
[(0.760000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'adaboost', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'none', 'feature_preprocessor:__choice__': 'random_trees_embedding', 'classifier:adaboost:algorithm': 'SAMME', 'classifier:adaboost:learning_

In [60]:
automl_predict=automl_ensemble_nbest_80.predict(x_test)
sklearn.metrics.accuracy_score(y_test, automl_predict)

0.66

In [62]:
automl_ensemble_size_80 = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,
    per_run_time_limit=30,
    ensemble_size=80
)
automl_ensemble_size_80.fit(x_train, y_train)
print('-------------------------------------------------------------------------------------------------------------------')
print(automl_ensemble_size_80.show_models())

[WARNING] [2021-07-20 04:33:24,746:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 25
[WARNING] [2021-07-20 04:33:27,860:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 12
[WARNING] [2021-07-20 04:33:36,656:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 6
-------------------------------------------------------------------------------------------------------------------
[(0.350000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': '

In [63]:
automl_predict=automl_ensemble_size_80.predict(x_test)
sklearn.metrics.accuracy_score(y_test, automl_predict)

0.6725

In [64]:
automl_run_time_50 = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,
    per_run_time_limit=50
)
automl_run_time_50.fit(x_train, y_train)
print('-------------------------------------------------------------------------------------------------------------------')
print(automl_run_time_50.show_models())

[WARNING] [2021-07-20 04:43:36,574:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 25
[WARNING] [2021-07-20 04:43:36,754:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 12
[WARNING] [2021-07-20 04:43:36,927:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 6
[WARNING] [2021-07-20 04:43:37,119:EnsembleBuilder(1):c49ad6282acc40e15000b558c6000afc] Memory Exception -- restart with less ensemble_nbest: 3
-------------------------------------------------------------------------------------------------------------------
[(0.480000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'da

In [65]:
automl_predict=automl_run_time_50.predict(x_test)
sklearn.metrics.accuracy_score(y_test, automl_predict)

0.6675